# Champion Classifier:
- match image data with excel file of champion labels
- background images
- VGG structure

- annotations_file (CSV)
    - file_name, one_hot_encoding_champion, champion_name 

- img_dir (Path)
    - path that contains all the image


In [4]:
# Packages
import numpy as np
import pandas as pd
import os
import shutil #copy and paste files
import re
import random
import pickle

### Read data

In [2]:
# Data of cropped images
df = pd.read_excel('data/Champions_Label.xlsx')
# Data of all champion names
master_champ_df = pd.read_excel('data/champion_list_master_dataset.xlsx')

# Columns to keep
columns = ['youtuber', 'video_name', 'frame_name', 'cropped_name', 'champion_name']

### Clean data

In [3]:
# list of all unique champions + background
master_champ_list = master_champ_df.champion_name.unique().tolist() + ['Background']

In [87]:
with open('data/master_champ_list.pkl', "wb") as output_file:
    pickle.dump(master_champ_list, output_file)

In [25]:
# Remove all NA values
df = df[~df.champion_name.isna()]

# Keep relevant columns
df = df.loc[:, columns]

# Check if names are all correct
df[~df.champion_name.isin(master_champ_list)]

,youtuber,video_name,frame_name,cropped_name,champion_name


In [27]:
# One hot encoding of champions
champion_df = df.champion_name.copy()
champion_df = champion_df.apply(lambda x: master_champ_list.index(x))

df['champion_onehot'] = champion_df

### Move all relevant images to data

In [94]:
def move_images(final_data_path, df, img_batch_name = ''):

    #final_data_path = os.path.join(os.getcwd(), 'data', 'champion-classifier')

    if not os.path.exists(final_data_path):
        os.makedirs(final_data_path)

    image_name_list = []
    image_path_list = []

    for i in range(len(df)):
        
        row = df.iloc[i, :]
        path_to_img = os.path.join(os.getcwd(), 'contents', row.youtuber, row.video_name, re.sub(r'.jpg', '', row.frame_name), re.sub(r'.jpg', '', row.cropped_name)+'.png')

        image_name = f'image_{i}_{img_batch_name}.jpg'
        path_to_final_img = os.path.join(final_data_path, image_name)
        image_name_list.append(image_name)
        image_path_list.append(path_to_final_img)
        shutil.copyfile(path_to_img, path_to_final_img)

    df['image_name'] = image_name_list
    df['image_file_path'] = image_path_list


    return df

            


In [33]:
move_images(final_data_path = os.path.join(os.getcwd(), 'data', 'champion-classifier'), df = df)

annotation_file = df.loc[:, ['image_name', 'champion_onehot', 'champion_name']]

annotation_file = annotation_file.reset_index(drop=True)

annotation_file.to_csv(os.path.join(os.getcwd(), 'data', 'annotation_file.csv'))

annotation_file.sample(10)

,image_name,champion_onehot,champion_name
35,image_35.jpg,4,Ezreal
832,image_832.jpg,46,Orianna
506,image_506.jpg,30,Leona
819,image_819.jpg,39,Ahri
473,image_473.jpg,31,Lucian
374,image_374.jpg,68,Tristana
253,image_253.jpg,79,Sion
25,image_25.jpg,63,Twisted Fate
282,image_282.jpg,83,Yuumi
770,image_770.jpg,35,Senna


# Create batch images (different from original training data)

In [133]:
# Data of cropped images
df = pd.read_excel('data/Champions_Label.xlsx')
# Data of all champion names
master_champ_df = pd.read_excel('data/champion_list_master_dataset.xlsx')

# Columns to keep
columns = ['youtuber', 'video_name', 'frame_name', 'cropped_name', 'champion_name']

# list of all unique champions + background
master_champ_list = master_champ_df.champion_name.unique().tolist() + ['Background']

In [134]:
df_unnamed = df[df.champion_name.isna()].reset_index(drop=True)

In [135]:
# DO NOT CHANGE THE ORDER OF THIS LIST:
video_list_check = ['100 HP Challenger - Nerfed but still so good  TFT Gizmos _ Gadgets  Teamfight Tactics.mp4',
                        '11 Loss Mercenary - 29 HP to 55 HP!  TFT Gizmos _ Gadgets  Teamfight Tactics.mp4']
                        # '3 Mutant Chogath - Augments are crazy!!  TFT Gizmos _ Gadgets  Teamfight Tactics.mp4',
                        # '4 Score Ahri - Gamble with Confidence  TFT Neon Nights  Teamfight Tactics.mp4',]


df_unnamed['champion_onehot'] = 0
df_unnamed['champion_name'] = 0

for i in range(len(video_list_check)):
    if i != 0:
        print(video_list_check[i])
        df_unamed_sample = df_unnamed[df_unnamed.video_name == video_list_check[i]]
        final_batch_data_path = os.path.join(os.getcwd(), 'data', f'batch-data-{i}')
        final_data_path = os.path.join(os.getcwd(), 'data', 'champion-classifier')

        move_images(final_batch_data_path, df_unamed_sample, img_batch_name=f'batch-data-{i}')
        move_images(final_data_path, df_unamed_sample, img_batch_name=f'batch-data-{i}')

        annotation_file = df_unamed_sample.loc[:, ['image_name', 'champion_onehot', 'champion_name']]

        annotation_file = annotation_file.reset_index(drop=True)

        annotation_file.to_csv(os.path.join(os.getcwd(), 'data', f'batch-data-{i}', f'annotation_file-{i}.csv'), index=False)


11 Loss Mercenary - 29 HP to 55 HP!  TFT Gizmos _ Gadgets  Teamfight Tactics.mp4


/var/folders/f7/7wm8ypsx1_zcy_df6_f5nrl40000gn/T/ipykernel_4950/1645491115.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['image_name'] = image_name_list
/var/folders/f7/7wm8ypsx1_zcy_df6_f5nrl40000gn/T/ipykernel_4950/1645491115.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['image_file_path'] = image_path_list


## Change the onehot to the proper one:
- after manually editing the annotation_file_final-i.csv

In [145]:
# CHANGE THIS
i = 1

annotation_final = pd.read_csv('data/annotation_final_file.csv')
annotation_batch = pd.read_csv(f'data/batch-data-{i}/annotation_file_final-{i}.csv')

# Series representing the onehot representation
champion_df = annotation_batch.champion_name.apply(lambda x: master_champ_list.index(x))

annotation_batch.champion_onehot = champion_df

annotation_batch

,image_name,champion_onehot,champion_name
0,image_0_batch-data-1.jpg,85,Background
1,image_1_batch-data-1.jpg,85,Background
2,image_2_batch-data-1.jpg,85,Background
3,image_3_batch-data-1.jpg,85,Background
4,image_4_batch-data-1.jpg,85,Background
...,...,...,...
1634,image_1634_batch-data-1.jpg,85,Background
1635,image_1635_batch-data-1.jpg,64,Graves
1636,image_1636_batch-data-1.jpg,7,Kassadin
1637,image_1637_batch-data-1.jpg,85,Background


In [146]:
test = pd.concat([annotation_final.loc[:,['image_name', 'champion_onehot', 'champion_name']], annotation_batch.loc[:,['image_name', 'champion_onehot', 'champion_name']]])

test.to_csv('data/annotation_final_file.csv', index = False)

In [48]:
# Delete later
data = pd.read_csv('data/annotation_final_file.csv')

In [49]:
final_data = data[data.champion_name != 'Background']


final_data.to_csv('data/annotation_final_file_champ_only.csv' , index = False)

final_data




,image_name,champion_onehot,champion_name
0,image_0.jpg,23,Warwick
1,image_1.jpg,74,Taric
2,image_2.jpg,2,Camille
9,image_9.jpg,22,Talon
10,image_10.jpg,73,Shaco
...,...,...,...
3929,image_1604_batch-data-1.jpg,17,Quinn
3942,image_1617_batch-data-1.jpg,1,Caitlyn
3953,image_1628_batch-data-1.jpg,69,Trundle
3960,image_1635_batch-data-1.jpg,64,Graves


In [50]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    final_data['image_name'],
    final_data['champion_onehot'],
    test_size = 0.25,
    random_state = 0,
    stratify=final_data['champion_onehot']
)

In [53]:
y_train

361     75
3690    74
123     83
551      7
496     30
        ..
91      30
704      5
69      72
2788     2
3843    69
Name: champion_onehot, Length: 878, dtype: int64